In [1]:
import numpy as np
import pandas as pd
import re
from rich import print
from ast import literal_eval

In [2]:
df_nonteencode = pd.read_csv("../data/seed_nonteencode.csv", converters={"alignments": literal_eval, "incorrect_alignments": literal_eval})
df_teencode = pd.read_csv("../data/seed_teencode.csv", converters={"alignments": literal_eval})
df_generated = pd.read_json("../data/fourth_generated.json")

df_generated

,vietnamese,english,alignments,incorrect_alignments,fix
0,Mày làm ăn như vậy thì ai mà chịu nổi,Who can stand you if you keep doing like that,"[Mày - You, làm ăn như vậy - doing like that, ...","[Mày - Who, ai mà - doing like]","[Mày - You, ai mà - Who can]"
1,"Mày làm ơn đi, tao chán mày lắm rồi","Please, I'm so fed up with you","[Mày - You, làm ơn đi - Please, tao chán mày l...","[làm ơn đi - I'm so fed up, Mày - Please]","[làm ơn đi - Please, Mày - You]"
2,Bữa nay tao chả muốn nói chuyện với mày,I don't feel like talking to you today,"[Bữa nay - today, tao chả muốn - I don't feel ...","[Bữa nay - talking, tao - you]","[Bữa nay - today, tao - I]"
3,Tao chả hỉu mày đang nói gì hết trơn,I have no idea what you're talking about,"[Tao - I, chả hỉu - have no idea, mày đang nói...","[Tao - have no idea, chả hỉu - I]","[Tao - I, chả hỉu - have no idea]"
4,Mày có nhớ lúc tao cứu mày khỏi bọn nó không?,Do you remember when I saved you from them?,"[Mày - You, có nhớ - remember, lúc tao - when ...","[lúc tao - I saved, bọn nó - you]","[lúc tao - when I, bọn nó - them]"
...,...,...,...,...,...
689,"Đừng có giả nai với tớ, cậu đấy.","Quit playing innocent with me, dude.","{'Đừng có giả nai': 'Quit playing innocent', '...","{'giả nai': 'innocent', 'với tớ': 'me'}","{'giả nai': 'playing innocent', 'với tớ': 'wit..."
690,"Sao mày lại bơ tao vậy, hả?","Why are you ghosting me, huh?","{'Sao mày lại': 'Why are you', 'bơ tao vậy': '...","{'bơ tao': 'ghosting', 'vậy': 'me'}","{'bơ tao': 'ghosting me', 'vậy': 'huh'}"
691,"Trời ơi, tớ stress quá!","Oh my God, I'm so stressed out!","{'Trời ơi': 'Oh my God', 'tớ stress quá': 'I'm...","{'tớ stress': 'I'm stressed', 'quá': 'out'}","{'tớ stress': 'I'm so stressed', 'quá': 'out'}"
692,Cậu ấy đang tán gẫu với bạn gái.,He's chatting up his girlfriend.,"{'Cậu ấy đang': 'He's', 'tán gẫu với': 'chatti...","{'tán gẫu': 'chatting', 'với': 'up'}","{'tán gẫu': 'chatting up', 'với': 'his girlfri..."


In [3]:
alignment_split_pattern = re.compile(r"\s*-\s*")

def generate_mt_instruct(x: str, y: str):
    template_prompt = """Dịch câu sau từ tiếng Việt sang tiếng Anh: 
Tiếng Việt: {}
Tiếng Anh: """

    prompt = template_prompt.format(x)
    return {"instruction": prompt, "response": y, "vietnamese": x, "english": y, "category": "translation"}

def generate_align_instruct(x: str, y: str, alignments: list[str], output: str):
    template_prompt = """Cho cặp câu tiếng Việt và tiếng Anh sau, hãy cho biết nhận định sau đây có đúng hay không:
Tiếng Việt: {}
Tiếng Anh: {}
Nhận định: "{}" có thể được dịch là "{}" """

    instructions = []
    

    for alignment in alignments:
        
        vi_alignment, en_alignment = alignment_split_pattern.split(alignment)

        prompt = template_prompt.format(x, y, vi_alignment, en_alignment)
        
        instructions.append({
            "instruction": prompt,
            "response": output,"vietnamese": x, "english": y,
            "category": "classification"
        })
        
    return instructions

def generate_hint_instruct(x: str, y: str, alignments: list[str]):
    template_prompt = """Sử dụng các gợi ý căn chỉnh sau và dịch từ tiếng Việt sang tiếng Anh
Căn chỉnh từ tiếng Việt sang tiếng Anh: 
{}
Tiếng Việt: {}
Tiếng Anh: """

    alignment_str = ""
    
    for alignment in alignments:
        alignment_line = ", ".join(alignment_split_pattern.split(alignment))
        
        alignment_str += f"- ({alignment_line})\n"


    prompt = template_prompt.format(alignment_str, x)
    
    return {"instruction": prompt, "response": y, "vietnamese": x, "english": y, "category": "translation_hint"}

def generate_revise_instruct(x: str, y: str, correct_alignments: list[str], incorrect_alignments: list[str], fix_alignments: list[str]):
    template_prompt = """Cho bản dịch tiếng Anh từ tiếng Việt sau, hãy cho biết các từ nào bị dịch sai và sửa lại chúng:
Tiếng Việt: {}
Tiếng Anh: {}"""

    wrong = y

    for correct_alignment, incorrect_alignment in zip(correct_alignments, incorrect_alignments):
        incorrect_english = alignment_split_pattern.split(incorrect_alignment)[1]
        correct_english = alignment_split_pattern.split(correct_alignment)[1]
        
        wrong = re.sub(correct_english, incorrect_english, y, flags=re.IGNORECASE)
        
        
    prompt = template_prompt.format(x, wrong)
    
    target = f"""Các từ bị dịch sai: {", ".join([alignment_split_pattern.split(alignment)[1] for alignment in incorrect_alignments])}. Sửa lại chúng thành các từ sau: {", ".join([alignment_split_pattern.split(alignment)[1] for alignment in fix_alignments])}. Câu được sửa lại: {y}"""
    
    return {"instruction": prompt, "response": target, "vietnamese": x, "english": y, "category": "translation_revision"}

def generate_instruction(example):
    mt_instruct = generate_mt_instruct(example["vietnamese"], example["english"])
    
    align_instructs_correct = generate_align_instruct(example["vietnamese"], example["english"], example["alignments"], "Đúng")
    
    
    hint_instruct = generate_hint_instruct(example["vietnamese"], example["english"], example["alignments"])
    
    if "fix" in example:
        align_instructions_incorrect = generate_align_instruct(example["vietnamese"], example["english"], example["incorrect_alignments"], "Sai")
        
        revise_instruct = generate_revise_instruct(example["vietnamese"], example["english"], example["alignments"], example["incorrect_alignments"], example["fix"])
        
        return [mt_instruct, *align_instructs_correct, hint_instruct, *align_instructions_incorrect, revise_instruct]
    
    return [mt_instruct, *align_instructs_correct, hint_instruct]
        

In [5]:
from math import e


instructions_data = []
error_count = 0

for i, example in df_generated.iterrows():
    try:
        instructions = generate_instruction(example)
    except:
        error_count += 1
        continue
    
    for instruction in instructions:
        instruction["metadata"] = "generated"
    
    instructions_data.extend(instructions)
    
for i, example in df_teencode.iterrows():
    try:
        instructions = generate_instruction(example)
    except:
        error_count += 1
        continue
    
    for instruction in instructions:
        instruction["metadata"] = "teencode"
    
    instructions_data.extend(instructions)
    
for i, example in df_nonteencode.iterrows():
    instructions = generate_instruction(example)
    
    for instruction in instructions:
        instruction["metadata"] = "nonteencode"
    
    instructions_data.extend(instructions)
    
df_instructions = pd.DataFrame(instructions_data)

df_instructions.to_json("../data/instructions.jsonl", orient="records", lines=True, force_ascii=False)
df_instructions

,instruction,response,vietnamese,english,category,metadata
0,Dịch câu sau từ tiếng Việt sang tiếng Anh: \nT...,Who can stand you if you keep doing like that,Mày làm ăn như vậy thì ai mà chịu nổi,Who can stand you if you keep doing like that,translation,generated
1,"Cho cặp câu tiếng Việt và tiếng Anh sau, hãy c...",Đúng,Mày làm ăn như vậy thì ai mà chịu nổi,Who can stand you if you keep doing like that,classification,generated
2,"Cho cặp câu tiếng Việt và tiếng Anh sau, hãy c...",Đúng,Mày làm ăn như vậy thì ai mà chịu nổi,Who can stand you if you keep doing like that,classification,generated
3,"Cho cặp câu tiếng Việt và tiếng Anh sau, hãy c...",Đúng,Mày làm ăn như vậy thì ai mà chịu nổi,Who can stand you if you keep doing like that,classification,generated
4,Sử dụng các gợi ý căn chỉnh sau và dịch từ tiế...,Who can stand you if you keep doing like that,Mày làm ăn như vậy thì ai mà chịu nổi,Who can stand you if you keep doing like that,translation_hint,generated
...,...,...,...,...,...,...
8209,"Cho cặp câu tiếng Việt và tiếng Anh sau, hãy c...",Đúng,tôi kiên trì khắc phục vấn đề đó,i keep trying to make it work,classification,nonteencode
8210,Sử dụng các gợi ý căn chỉnh sau và dịch từ tiế...,i keep trying to make it work,tôi kiên trì khắc phục vấn đề đó,i keep trying to make it work,translation_hint,nonteencode
8211,Dịch câu sau từ tiếng Việt sang tiếng Anh: \nT...,these senior water rights are key,điểm mấu chốt ở đây là những quyền ưu tiên đó,these senior water rights are key,translation,nonteencode
8212,"Cho cặp câu tiếng Việt và tiếng Anh sau, hãy c...",Đúng,điểm mấu chốt ở đây là những quyền ưu tiên đó,these senior water rights are key,classification,nonteencode


In [119]:
example = df_teencode.iloc[0]
print(example)

print(generate_instruction(example))
# mt_instruct = generate_mt_instruct(example["vietnamese"], example["english"])
# print(mt_instruct)
# align_instructs_correct = generate_align_instruct(example["vietnamese"], example["english"], example["alignments"], "Đúng")
# align_instructions_incorrect = generate_align_instruct(example["vietnamese"], example["english"], example["incorrect_alignments"], "Sai")

# print("-"*100)
# for align_instruct in align_instructs_correct:
#     print(align_instruct)

# for align_instruct in align_instructions_incorrect:
#     print(align_instruct)
    
# print("-"*100)

# hint_instruct = generate_hint_instruct(example["vietnamese"], example["english"], example["alignments"])
# print(hint_instruct)

# print("-"*100)
# revise_instruct = generate_revise_instruct(example["vietnamese"], example["english"], example["alignments"], example["incorrect_alignments"], example["fix"])
# print(revise_instruct)

vietnamese    Tính tao tao biết, chẳng có chuyện gì có thể l...
english            I know how I tick, nothing can drive me nuts
alignments    [Tính tao - how I tick, phát điên - drive me n...
Name: 0, dtype: object

[
    {
        'prompt': 'Dịch câu sau từ tiếng Việt sang tiếng Anh: \nTiếng Việt: Tính tao tao biết, chẳng có chuyện gì 
có thể làm tao phát điên cả\nTiếng Anh: ',
        'target': 'I know how I tick, nothing can drive me nuts'
    },
    {
        'prompt': 'Cho cặp câu tiếng Việt và tiếng Anh sau, hãy cho biết nhận định sau đây có đúng hay 
không:\nTiếng Việt: Tính tao tao biết, chẳng có chuyện gì có thể làm tao phát điên cả\nTiếng Anh: I know how I 
tick, nothing can drive me nuts\nNhận định: "Tính tao" có thể được dịch là "how I tick" ',
        'target': 'Đúng'
    },
    {
        'prompt': 'Cho cặp câu tiếng Việt và tiếng Anh sau, hãy cho biết nhận định sau đây có đúng hay 
không:\nTiếng Việt: Tính tao tao biết, chẳng có chuyện gì có thể làm tao phát điên cả\nTiếng Anh: I know how I 
tick, nothing can drive me nuts\nNhận định: "phát điên" có thể được dịch là "drive me nuts" ',
        'target': 'Đúng'
    },
    {
        'prompt': 'Sử dụng các gợi ý căn chỉnh sau và dịch từ tiếng Việt sang tiếng Anh\nCăn chỉnh từ tiếng Việt 
sang tiếng Anh: \n- (Tính tao, how I tick)\n- (phát điên, drive me nuts)\n\nTiếng Việt: Tính tao tao biết, chẳng có
chuyện gì có thể làm tao phát điên cả\nTiếng Anh: ',
        'target': 'I know how I tick, nothing can drive me nuts'
    }
]